In [1]:
#before running this please change the RUNTIME to GPU (Runtime -> Change runtime type -> set harware accelarotor as GPU)
#Mount our google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Note : only needed when you have to download the processed data to the environment
#download and unzip the data from google drive Colab environment
from google_drive_downloader import GoogleDriveDownloader as gdd

# https://drive.google.com/drive/folders/1hlagmkHvVahBBb_EJv8vhUOsZWoPYgS6?usp=share_link --------Faceforensics
# https://drive.google.com/drive/folders/1h36Be3S1Mt4tkItIHakNAReUOnLoGXOi?usp=drive_link  ------- whole data drive
#use only file id of the link
#Note: Below link is just an example, Not an actual link. Actual Links are in ReadMe file
#https://drive.google.com/file/d/1ubvKLzBDe5i1acxgGUK6ObeNBYCKUS07/view?usp=sharing
# url = '1ubvKLzBDe5i1acxgGUK6ObeNBYCKUS07'   #Like this

url = '1h36Be3S1Mt4tkItIHakNAReUOnLoGXOi'
gdd.download_file_from_google_drive(file_id = url,dest_path='./data.zip',unzip=True)


Unzipping...

/usr/local/lib/python3.10/dist-packages/google_drive_downloader/google_drive_downloader.py:78: UserWarning: Ignoring `unzip` since "1h36Be3S1Mt4tkItIHakNAReUOnLoGXOi" does not look like a valid zip file
  warnings.warn('Ignoring `unzip` since "{}" does not look like a valid zip file'.format(file_id))


In [3]:
#to load preprocessod video to memory
import json
import glob
import numpy as np
import cv2
import copy
import random
video_files =  glob.glob('/content/drive/My Drive/Data/Celeb_fake_face_only/*.mp4')
video_files += glob.glob('/content/drive/My Drive/Data/Celeb_real_face_only/*.mp4')
# video_files += glob.glob('/content/drive/My Drive/DFDC_FAKE_Face_only_data/*.mp4')
# video_files += glob.glob('/content/drive/My Drive/DFDC_REAL_Face_only_data/*.mp4')
# video_files += glob.glob('/content/drive/My Drive/FF_Face_only_data/*.mp4')

# video_files = glob.glob('/content/drive/My Drive/Data/FF_Face_only_data/*.mp4')

random.shuffle(video_files)
random.shuffle(video_files)
frame_count = []
for video_file in video_files:
  cap = cv2.VideoCapture(video_file)
  if(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))<100):
    video_files.remove(video_file)
    continue
  frame_count.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
print("frames are " , frame_count)
print("Total no of video: " , len(frame_count))
print('Average frame per video:',np.mean(frame_count))

frames are  [148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 124, 148, 148, 148, 148, 148, 14

In [1]:
# For this notebook to run with updated APIs, we need torch 1.12+ and torchvision 0.13+
try:
    import torch
    import torchvision
    assert int(torch.__version__.split(".")[1]) >= 12, "torch version should be 1.12+"
    assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version should be 0.13+"
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")
except:
    print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")
    !pip3 install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
    import torch
    import torchvision
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")

[INFO] torch/torchvision versions not as required, installing nightly versions.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu113
torch version: 2.0.1+cu118
torchvision version: 0.15.2+cu118


In [2]:
# Continue with regular imports
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

# Try to import the going_modular directory, download it from GitHub if it doesn't work
try:
    from going_modular.going_modular import data_setup, engine
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !rm -rf pytorch-deep-learning
    from going_modular.going_modular import data_setup, engine

In [3]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [6]:
#Model with feature visualization
from torch import nn
from torchvision import models
class Model(nn.Module):
    def __init__(self, num_classes,latent_dim= 2048, lstm_layers=1 , hidden_dim = 2048, bidirectional = False):
        super(Model, self).__init__()
        model = models.resnext50_32x4d(pretrained = True) #Residual Network CNN
        # wts =
        # model = models.resnext50_32x4d(wts) #Residual Network CNN
        # weights = torchvision.models.EfficientNet_B0_Weights.IMAGENET1K_V1
        # model = models.efficientnet_b0(weights)
        self.model = nn.Sequential(*list(model.children())[:-2])
        self.lstm = nn.LSTM(latent_dim,hidden_dim, lstm_layers,  bidirectional)
        self.relu = nn.LeakyReLU()
        self.dp = nn.Dropout(0.4)
        self.linear1 = nn.Linear(2048,num_classes)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
    def forward(self, x):
        batch_size,seq_length, c, h, w = x.shape
        x = x.view(batch_size * seq_length, c, h, w)
        fmap = self.model(x)
        x = self.avgpool(fmap)
        x = x.view(batch_size,seq_length,2048)
        x_lstm,_ = self.lstm(x,None)
        return fmap,self.dp(self.linear1(torch.mean(x_lstm,dim = 1)))

In [4]:
import numpy as np

In [20]:
model_efficientnet = Model(2)
a,b = model_efficientnet(torch.from_numpy(np.empty((1,20,3,112,112))).type(torch.FloatTensor))

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt50_32X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [22]:
from torchinfo import summary

# summary(model, input_size=(batch_size, 1, 28, 28))
summary(model_efficientnet, input_size=((1,20,3,112,112)))

Layer (type:depth-idx)                        Output Shape              Param #
Model                                         [20, 2048, 4, 4]          --
├─Sequential: 1-1                             [20, 2048, 4, 4]          --
│    └─Conv2d: 2-1                            [20, 64, 56, 56]          9,408
│    └─BatchNorm2d: 2-2                       [20, 64, 56, 56]          128
│    └─ReLU: 2-3                              [20, 64, 56, 56]          --
│    └─MaxPool2d: 2-4                         [20, 64, 28, 28]          --
│    └─Sequential: 2-5                        [20, 256, 28, 28]         --
│    │    └─Bottleneck: 3-1                   [20, 256, 28, 28]         63,488
│    │    └─Bottleneck: 3-2                   [20, 256, 28, 28]         71,168
│    │    └─Bottleneck: 3-3                   [20, 256, 28, 28]         71,168
│    └─Sequential: 2-6                        [20, 512, 14, 14]         --
│    │    └─Bottleneck: 3-4                   [20, 512, 14, 14]         349,184

In [58]:
m1 = models.efficientnet_b1(weights="IMAGENET1K_V1")
for param in m1.features.parameters():
    param.requires_grad = False
m1.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True),
    torch.nn.Linear(in_features=1280,
                    out_features=2, # same number of output units as our number of classes
                    bias=True)).to(device)

summary(m1, input_size=((4,3,112,112)), col_names=["input_size", "output_size", "num_params", "trainable"],)
# summary(m1)
# m1.state_dict()

Layer (type:depth-idx)                                  Input Shape               Output Shape              Param #                   Trainable
EfficientNet                                            [4, 3, 112, 112]          [4, 2]                    --                        Partial
├─Sequential: 1-1                                       [4, 3, 112, 112]          [4, 1280, 4, 4]           --                        False
│    └─Conv2dNormActivation: 2-1                        [4, 3, 112, 112]          [4, 32, 56, 56]           --                        False
│    │    └─Conv2d: 3-1                                 [4, 3, 112, 112]          [4, 32, 56, 56]           (864)                     False
│    │    └─BatchNorm2d: 3-2                            [4, 32, 56, 56]           [4, 32, 56, 56]           (64)                      False
│    │    └─SiLU: 3-3                                   [4, 32, 56, 56]           [4, 32, 56, 56]           --                        --
│    └─Sequential

In [57]:
m0 = models.efficientnet_b0(weights="IMAGENET1K_V1")
# m0.classifier = torch.nn.Sequential(
#     torch.nn.Dropout(p=0.2, inplace=True),
#     torch.nn.Linear(in_features=1280,
#                     out_features=2, # same number of output units as our number of classes
#                     bias=True)).to(device)

summary(m0, input_size=((4,3,112,112)), col_names=["input_size", "output_size", "num_params", "trainable"],)

Layer (type:depth-idx)                                  Input Shape               Output Shape              Param #                   Trainable
EfficientNet                                            [4, 3, 112, 112]          [4, 1000]                 --                        True
├─Sequential: 1-1                                       [4, 3, 112, 112]          [4, 1280, 4, 4]           --                        True
│    └─Conv2dNormActivation: 2-1                        [4, 3, 112, 112]          [4, 32, 56, 56]           --                        True
│    │    └─Conv2d: 3-1                                 [4, 3, 112, 112]          [4, 32, 56, 56]           864                       True
│    │    └─BatchNorm2d: 3-2                            [4, 32, 56, 56]           [4, 32, 56, 56]           64                        True
│    │    └─SiLU: 3-3                                   [4, 32, 56, 56]           [4, 32, 56, 56]           --                        --
│    └─Sequential: 2-2  

In [8]:
m1_vgg = models.vgg16(weights="IMAGENET1K_V1")
summary(m1_vgg, input_size=((4,3,224,224)), col_names=["input_size", "output_size", "num_params", "trainable"])

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Trainable
VGG                                      [4, 3, 224, 224]          [4, 1000]                 --                        True
├─Sequential: 1-1                        [4, 3, 224, 224]          [4, 512, 7, 7]            --                        True
│    └─Conv2d: 2-1                       [4, 3, 224, 224]          [4, 64, 224, 224]         1,792                     True
│    └─ReLU: 2-2                         [4, 64, 224, 224]         [4, 64, 224, 224]         --                        --
│    └─Conv2d: 2-3                       [4, 64, 224, 224]         [4, 64, 224, 224]         36,928                    True
│    └─ReLU: 2-4                         [4, 64, 224, 224]         [4, 64, 224, 224]         --                        --
│    └─MaxPool2d: 2-5                    [4, 64, 224, 224]         [4, 64, 112, 112]         --                        --
│    └─Co

In [26]:
for param in m1_vgg.features.parameters():
    param.requires_grad = False


m1_vgg.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2),
    torch.nn.Linear(in_features=25088,
                    out_features=2, # same number of output units as our number of classes
                    bias=True)).to(device)

summary(m1_vgg, input_size=((4,3,224,224)), col_names=["input_size", "output_size", "num_params", "trainable"])

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Trainable
VGG                                      [4, 3, 224, 224]          [4, 2]                    --                        Partial
├─Sequential: 1-1                        [4, 3, 224, 224]          [4, 512, 7, 7]            --                        False
│    └─Conv2d: 2-1                       [4, 3, 224, 224]          [4, 64, 224, 224]         (1,792)                   False
│    └─ReLU: 2-2                         [4, 64, 224, 224]         [4, 64, 224, 224]         --                        --
│    └─Conv2d: 2-3                       [4, 64, 224, 224]         [4, 64, 224, 224]         (36,928)                  False
│    └─ReLU: 2-4                         [4, 64, 224, 224]         [4, 64, 224, 224]         --                        --
│    └─MaxPool2d: 2-5                    [4, 64, 224, 224]         [4, 64, 112, 112]         --                        --
│  

In [7]:
eff2 = models.efficientnet_b2(weights="IMAGENET1K_V1")
print(f"Number of in_features to final layer of EfficientNetB2: {len(eff2.classifier.state_dict()['1.weight'][0])}")

Downloading: "https://download.pytorch.org/models/efficientnet_b2_rwightman-bcdf34b7.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b2_rwightman-bcdf34b7.pth
100%|██████████| 35.2M/35.2M [00:00<00:00, 41.6MB/s]


Number of in_features to final layer of EfficientNetB2: 1408


In [9]:
eff3 = models.efficientnet_b3(weights="IMAGENET1K_V1")
print(f"Number of in_features to final layer of EfficientNetB3: {len(eff3.classifier.state_dict()['1.weight'][0])}")

Number of in_features to final layer of EfficientNetB3: 1536


In [8]:
eff4 = models.efficientnet_b4(weights="IMAGENET1K_V1")
print(f"Number of in_features to final layer of EfficientNetB4: {len(eff4.classifier.state_dict()['1.weight'][0])}")

Downloading: "https://download.pytorch.org/models/efficientnet_b4_rwightman-7eb33cd5.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b4_rwightman-7eb33cd5.pth
100%|██████████| 74.5M/74.5M [00:00<00:00, 91.1MB/s]


Number of in_features to final layer of EfficientNetB4: 1792


In [10]:
eff5 = models.efficientnet_b5(weights="IMAGENET1K_V1")
print(f"Number of in_features to final layer of EfficientNetB5: {len(eff5.classifier.state_dict()['1.weight'][0])}")

Downloading: "https://download.pytorch.org/models/efficientnet_b5_lukemelas-b6417697.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b5_lukemelas-b6417697.pth
100%|██████████| 117M/117M [00:01<00:00, 87.9MB/s]


Number of in_features to final layer of EfficientNetB5: 2048


In [12]:
eff6 = models.efficientnet_b6(weights="IMAGENET1K_V1")
print(f"Number of in_features to final layer of EfficientNetB6: {len(eff6.classifier.state_dict()['1.weight'][0])}")

Number of in_features to final layer of EfficientNetB6: 2304


In [13]:
eff7 = models.efficientnet_b7(weights="IMAGENET1K_V1")
print(f"Number of in_features to final layer of EfficientNetB7: {len(eff7.classifier.state_dict()['1.weight'][0])}")

Downloading: "https://download.pytorch.org/models/efficientnet_b7_lukemelas-dcc49843.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b7_lukemelas-dcc49843.pth
100%|██████████| 255M/255M [00:02<00:00, 90.7MB/s]


Number of in_features to final layer of EfficientNetB7: 2560


In [21]:
m_vgg11 = models.vgg11(weights="IMAGENET1K_V1")
summary(m_vgg11, input_size=((4,3,112,112)), col_names=["input_size", "output_size", "num_params", "trainable"])

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Trainable
VGG                                      [4, 3, 112, 112]          [4, 1000]                 --                        True
├─Sequential: 1-1                        [4, 3, 112, 112]          [4, 512, 3, 3]            --                        True
│    └─Conv2d: 2-1                       [4, 3, 112, 112]          [4, 64, 112, 112]         1,792                     True
│    └─ReLU: 2-2                         [4, 64, 112, 112]         [4, 64, 112, 112]         --                        --
│    └─MaxPool2d: 2-3                    [4, 64, 112, 112]         [4, 64, 56, 56]           --                        --
│    └─Conv2d: 2-4                       [4, 64, 56, 56]           [4, 128, 56, 56]          73,856                    True
│    └─ReLU: 2-5                         [4, 128, 56, 56]          [4, 128, 56, 56]          --                        --
│    └─Ma

In [22]:
m_vgg16 = models.vgg16(weights="IMAGENET1K_V1")
summary(m_vgg16, input_size=((4,3,112,112)), col_names=["input_size", "output_size", "num_params", "trainable"])

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Trainable
VGG                                      [4, 3, 112, 112]          [4, 1000]                 --                        True
├─Sequential: 1-1                        [4, 3, 112, 112]          [4, 512, 3, 3]            --                        True
│    └─Conv2d: 2-1                       [4, 3, 112, 112]          [4, 64, 112, 112]         1,792                     True
│    └─ReLU: 2-2                         [4, 64, 112, 112]         [4, 64, 112, 112]         --                        --
│    └─Conv2d: 2-3                       [4, 64, 112, 112]         [4, 64, 112, 112]         36,928                    True
│    └─ReLU: 2-4                         [4, 64, 112, 112]         [4, 64, 112, 112]         --                        --
│    └─MaxPool2d: 2-5                    [4, 64, 112, 112]         [4, 64, 56, 56]           --                        --
│    └─Co

In [24]:
m_xcep = models.googlenet(weights="IMAGENET1K_V1")
summary(m_xcep, input_size=((4,3,112,112)), col_names=["input_size", "output_size", "num_params", "trainable"])

Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/hub/checkpoints/googlenet-1378be20.pth
100%|██████████| 49.7M/49.7M [00:00<00:00, 90.2MB/s]


Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Trainable
GoogLeNet                                [4, 3, 112, 112]          [4, 1000]                 --                        True
├─BasicConv2d: 1-1                       [4, 3, 112, 112]          [4, 64, 56, 56]           --                        True
│    └─Conv2d: 2-1                       [4, 3, 112, 112]          [4, 64, 56, 56]           9,408                     True
│    └─BatchNorm2d: 2-2                  [4, 64, 56, 56]           [4, 64, 56, 56]           128                       True
├─MaxPool2d: 1-2                         [4, 64, 56, 56]           [4, 64, 28, 28]           --                        --
├─BasicConv2d: 1-3                       [4, 64, 28, 28]           [4, 64, 28, 28]           --                        True
│    └─Conv2d: 2-3                       [4, 64, 28, 28]           [4, 64, 28, 28]           4,096                     True
│    

In [34]:
m_xcep.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2),
    torch.nn.Linear(in_features=1024,
                    out_features=2, # same number of output units as our number of classes
                    bias=True)).to(device)

summary(m_xcep, input_size=((4,3,224,224)), col_names=["input_size", "output_size", "num_params", "trainable"])

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Trainable
GoogLeNet                                [4, 3, 224, 224]          [4, 1000]                 2,050                     True
├─BasicConv2d: 1-1                       [4, 3, 224, 224]          [4, 64, 112, 112]         --                        True
│    └─Conv2d: 2-1                       [4, 3, 224, 224]          [4, 64, 112, 112]         9,408                     True
│    └─BatchNorm2d: 2-2                  [4, 64, 112, 112]         [4, 64, 112, 112]         128                       True
├─MaxPool2d: 1-2                         [4, 64, 112, 112]         [4, 64, 56, 56]           --                        --
├─BasicConv2d: 1-3                       [4, 64, 56, 56]           [4, 64, 56, 56]           --                        True
│    └─Conv2d: 2-3                       [4, 64, 56, 56]           [4, 64, 56, 56]           4,096                     True
│    

In [29]:
m_conv_t = models.convnext_tiny(weights="IMAGENET1K_V1")
summary(m_conv_t, input_size=((4,3,112,112)), col_names=["input_size", "output_size", "num_params", "trainable"])

Layer (type:depth-idx)                        Input Shape               Output Shape              Param #                   Trainable
ConvNeXt                                      [4, 3, 112, 112]          [4, 1000]                 --                        True
├─Sequential: 1-1                             [4, 3, 112, 112]          [4, 768, 3, 3]            --                        True
│    └─Conv2dNormActivation: 2-1              [4, 3, 112, 112]          [4, 96, 28, 28]           --                        True
│    │    └─Conv2d: 3-1                       [4, 3, 112, 112]          [4, 96, 28, 28]           4,704                     True
│    │    └─LayerNorm2d: 3-2                  [4, 96, 28, 28]           [4, 96, 28, 28]           192                       True
│    └─Sequential: 2-2                        [4, 96, 28, 28]           [4, 96, 28, 28]           --                        True
│    │    └─CNBlock: 3-3                      [4, 96, 28, 28]           [4, 96, 28, 28]     

In [28]:
m_conv = models.convnext_small(weights="IMAGENET1K_V1")
summary(m_conv, input_size=((4,3,112,112)), col_names=["input_size", "output_size", "num_params", "trainable"])

Downloading: "https://download.pytorch.org/models/convnext_small-0c510722.pth" to /root/.cache/torch/hub/checkpoints/convnext_small-0c510722.pth
100%|██████████| 192M/192M [00:01<00:00, 130MB/s] 


Layer (type:depth-idx)                        Input Shape               Output Shape              Param #                   Trainable
ConvNeXt                                      [4, 3, 112, 112]          [4, 1000]                 --                        True
├─Sequential: 1-1                             [4, 3, 112, 112]          [4, 768, 3, 3]            --                        True
│    └─Conv2dNormActivation: 2-1              [4, 3, 112, 112]          [4, 96, 28, 28]           --                        True
│    │    └─Conv2d: 3-1                       [4, 3, 112, 112]          [4, 96, 28, 28]           4,704                     True
│    │    └─LayerNorm2d: 3-2                  [4, 96, 28, 28]           [4, 96, 28, 28]           192                       True
│    └─Sequential: 2-2                        [4, 96, 28, 28]           [4, 96, 28, 28]           --                        True
│    │    └─CNBlock: 3-3                      [4, 96, 28, 28]           [4, 96, 28, 28]     

**Chk Densenet all 4**

In [30]:
m_mob = models.mobilenet_v2(weights="IMAGENET1K_V1")
summary(m_mob, input_size=((4,3,112,112)), col_names=["input_size", "output_size", "num_params", "trainable"])

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 80.5MB/s]


Layer (type:depth-idx)                             Input Shape               Output Shape              Param #                   Trainable
MobileNetV2                                        [4, 3, 112, 112]          [4, 1000]                 --                        True
├─Sequential: 1-1                                  [4, 3, 112, 112]          [4, 1280, 4, 4]           --                        True
│    └─Conv2dNormActivation: 2-1                   [4, 3, 112, 112]          [4, 32, 56, 56]           --                        True
│    │    └─Conv2d: 3-1                            [4, 3, 112, 112]          [4, 32, 56, 56]           864                       True
│    │    └─BatchNorm2d: 3-2                       [4, 32, 56, 56]           [4, 32, 56, 56]           64                        True
│    │    └─ReLU6: 3-3                             [4, 32, 56, 56]           [4, 32, 56, 56]           --                        --
│    └─InvertedResidual: 2-2                       [4, 32, 

In [20]:
m_sqz0 = models.squeezenet1_0(weights="IMAGENET1K_V1")
summary(m_sqz0, input_size=((4,3,112,112)), col_names=["input_size", "output_size", "num_params", "trainable"])

Downloading: "https://download.pytorch.org/models/squeezenet1_0-b66bff10.pth" to /root/.cache/torch/hub/checkpoints/squeezenet1_0-b66bff10.pth
100%|██████████| 4.78M/4.78M [00:00<00:00, 67.9MB/s]


Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Trainable
SqueezeNet                               [4, 3, 112, 112]          [4, 1000]                 --                        True
├─Sequential: 1-1                        [4, 3, 112, 112]          [4, 512, 6, 6]            --                        True
│    └─Conv2d: 2-1                       [4, 3, 112, 112]          [4, 96, 53, 53]           14,208                    True
│    └─ReLU: 2-2                         [4, 96, 53, 53]           [4, 96, 53, 53]           --                        --
│    └─MaxPool2d: 2-3                    [4, 96, 53, 53]           [4, 96, 26, 26]           --                        --
│    └─Fire: 2-4                         [4, 96, 26, 26]           [4, 128, 26, 26]          --                        True
│    │    └─Conv2d: 3-1                  [4, 96, 26, 26]           [4, 16, 26, 26]           1,552                     True
│    │ 

In [19]:
m_sqz = models.squeezenet1_1(weights="IMAGENET1K_V1")
summary(m_sqz, input_size=((4,3,112,112)), col_names=["input_size", "output_size", "num_params", "trainable"])

Downloading: "https://download.pytorch.org/models/squeezenet1_1-b8a52dc0.pth" to /root/.cache/torch/hub/checkpoints/squeezenet1_1-b8a52dc0.pth
100%|██████████| 4.73M/4.73M [00:00<00:00, 72.1MB/s]


Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Trainable
SqueezeNet                               [4, 3, 112, 112]          [4, 1000]                 --                        True
├─Sequential: 1-1                        [4, 3, 112, 112]          [4, 512, 6, 6]            --                        True
│    └─Conv2d: 2-1                       [4, 3, 112, 112]          [4, 64, 55, 55]           1,792                     True
│    └─ReLU: 2-2                         [4, 64, 55, 55]           [4, 64, 55, 55]           --                        --
│    └─MaxPool2d: 2-3                    [4, 64, 55, 55]           [4, 64, 27, 27]           --                        --
│    └─Fire: 2-4                         [4, 64, 27, 27]           [4, 128, 27, 27]          --                        True
│    │    └─Conv2d: 3-1                  [4, 64, 27, 27]           [4, 16, 27, 27]           1,040                     True
│    │ 

In [18]:
m_incep = models.inception_v3(weights="IMAGENET1K_V1")
summary(m_incep, input_size=((4,3,112,112)), col_names=["input_size", "output_size", "num_params", "trainable"])

Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:01<00:00, 88.3MB/s]


Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Trainable
Inception3                               [4, 3, 112, 112]          [4, 1000]                 3,326,696                 True
├─BasicConv2d: 1-1                       [4, 3, 112, 112]          [4, 32, 55, 55]           --                        True
│    └─Conv2d: 2-1                       [4, 3, 112, 112]          [4, 32, 55, 55]           864                       True
│    └─BatchNorm2d: 2-2                  [4, 32, 55, 55]           [4, 32, 55, 55]           64                        True
├─BasicConv2d: 1-2                       [4, 32, 55, 55]           [4, 32, 53, 53]           --                        True
│    └─Conv2d: 2-3                       [4, 32, 55, 55]           [4, 32, 53, 53]           9,216                     True
│    └─BatchNorm2d: 2-4                  [4, 32, 53, 53]           [4, 32, 53, 53]           64                        True
├─B

In [75]:
mm = models.efficientnet_b3(weights="IMAGENET1K_V1")

In [78]:
# summary(model, input_size=(batch_size, 1, 28, 28))
summary(mm, input_size=((20,3,112,112)))

Layer (type:depth-idx)                                  Output Shape              Param #
EfficientNet                                            [20, 1000]                --
├─Sequential: 1-1                                       [20, 1536, 4, 4]          --
│    └─Conv2dNormActivation: 2-1                        [20, 40, 56, 56]          --
│    │    └─Conv2d: 3-1                                 [20, 40, 56, 56]          1,080
│    │    └─BatchNorm2d: 3-2                            [20, 40, 56, 56]          80
│    │    └─SiLU: 3-3                                   [20, 40, 56, 56]          --
│    └─Sequential: 2-2                                  [20, 24, 56, 56]          --
│    │    └─MBConv: 3-4                                 [20, 24, 56, 56]          2,298
│    │    └─MBConv: 3-5                                 [20, 24, 56, 56]          1,206
│    └─Sequential: 2-3                                  [20, 32, 28, 28]          --
│    │    └─MBConv: 3-6                            

In [6]:
from torch import nn
from torchvision import models
class Model(nn.Module):
    def __init__(self, num_classes,latent_dim= 1536, lstm_layers=1 , hidden_dim = 1536, bidirectional = False):
        super(Model, self).__init__()
        # model = models.resnext50_32x4d(pretrained = True) #Residual Network CNN
        # wts =
        # model = models.resnext50_32x4d(wts) #Residual Network CNN
        weights = torchvision.models.EfficientNet_B3_Weights.IMAGENET1K_V1
        model = models.efficientnet_b3(weights)
        self.model = nn.Sequential(*list(model.children())[:-2])
        self.lstm = nn.LSTM(latent_dim,hidden_dim, lstm_layers,  bidirectional)
        self.relu = nn.LeakyReLU()
        self.dp = nn.Dropout(0.4)
        self.linear1 = nn.Linear(1536,num_classes)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
    def forward(self, x):
        batch_size,seq_length, c, h, w = x.shape
        x = x.view(batch_size * seq_length, c, h, w)
        fmap = self.model(x)
        x = self.avgpool(fmap)
        x = x.view(batch_size,seq_length,1536)
        x_lstm,_ = self.lstm(x,None)
        return fmap,self.dp(self.linear1(torch.mean(x_lstm,dim = 1)))

In [9]:
mchk = Model(2)
a,b = mchk(torch.from_numpy(np.empty((1,20,3,112,112))).type(torch.FloatTensor))

In [10]:
from torchinfo import summary

# summary(model, input_size=(batch_size, 1, 28, 28))
summary(mchk, input_size=((1,20,3,112,112)))

Layer (type:depth-idx)                                       Output Shape              Param #
Model                                                        [20, 1536, 4, 4]          --
├─Sequential: 1-1                                            [20, 1536, 4, 4]          --
│    └─Sequential: 2-1                                       [20, 1536, 4, 4]          --
│    │    └─Conv2dNormActivation: 3-1                        [20, 40, 56, 56]          1,160
│    │    └─Sequential: 3-2                                  [20, 24, 56, 56]          3,504
│    │    └─Sequential: 3-3                                  [20, 32, 28, 28]          48,118
│    │    └─Sequential: 3-4                                  [20, 48, 14, 14]          110,912
│    │    └─Sequential: 3-5                                  [20, 96, 7, 7]            638,700
│    │    └─Sequential: 3-6                                  [20, 136, 7, 7]           1,387,760
│    │    └─Sequential: 3-7                                  [20, 23

In [6]:
#Model with feature visualization
from torch import nn
from torchvision import models
class Model(nn.Module):
    def __init__(self, num_classes,latent_dim= 2048, lstm_layers=1 , hidden_dim = 2048, bidirectional = False):
        super(Model, self).__init__()
        model = models.resnext50_32x4d(pretrained = True) #Residual Network CNN
        # wts =
        # model = models.resnext50_32x4d(wts) #Residual Network CNN
        # weights = torchvision.models.EfficientNet_B0_Weights.IMAGENET1K_V1
        # model = models.efficientnet_b0(weights)
        self.model = nn.Sequential(*list(model.children())[:-2])
        self.lstm = nn.LSTM(latent_dim,hidden_dim, lstm_layers,  bidirectional)
        self.relu = nn.LeakyReLU()
        self.dp = nn.Dropout(0.4)
        self.linear1 = nn.Linear(2048,num_classes)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
    def forward(self, x):
        batch_size,seq_length, c, h, w = x.shape
        x = x.view(batch_size * seq_length, c, h, w)
        fmap = self.model(x)
        x = self.avgpool(fmap)
        x = x.view(batch_size,seq_length,2048)
        x_lstm,_ = self.lstm(x,None)
        return fmap,self.dp(self.linear1(torch.mean(x_lstm,dim = 1)))

In [7]:
m_res_chk1 = Model(2)
a,b = m_res_chk1(torch.from_numpy(np.empty((1,20,3,112,112))).type(torch.FloatTensor))

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt50_32X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
summary(m_res_chk1, input_size=((1,20,3,112,112)), col_names=["input_size", "output_size", "num_params", "trainable"])

Layer (type:depth-idx)                        Input Shape               Output Shape              Param #                   Trainable
Model                                         [1, 20, 3, 112, 112]      [20, 2048, 4, 4]          --                        True
├─Sequential: 1-1                             [20, 3, 112, 112]         [20, 2048, 4, 4]          --                        True
│    └─Conv2d: 2-1                            [20, 3, 112, 112]         [20, 64, 56, 56]          9,408                     True
│    └─BatchNorm2d: 2-2                       [20, 64, 56, 56]          [20, 64, 56, 56]          128                       True
│    └─ReLU: 2-3                              [20, 64, 56, 56]          [20, 64, 56, 56]          --                        --
│    └─MaxPool2d: 2-4                         [20, 64, 56, 56]          [20, 64, 28, 28]          --                        --
│    └─Sequential: 2-5                        [20, 64, 28, 28]          [20, 256, 28, 28]       

In [18]:
m_res_chk2 = models.resnext50_32x4d(weights="IMAGENET1K_V1")
summary(m_res_chk2, input_size=((20,3,112,112)), col_names=["input_size", "output_size", "num_params", "trainable"])

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Trainable
ResNet                                   [20, 3, 112, 112]         [20, 1000]                --                        True
├─Conv2d: 1-1                            [20, 3, 112, 112]         [20, 64, 56, 56]          9,408                     True
├─BatchNorm2d: 1-2                       [20, 64, 56, 56]          [20, 64, 56, 56]          128                       True
├─ReLU: 1-3                              [20, 64, 56, 56]          [20, 64, 56, 56]          --                        --
├─MaxPool2d: 1-4                         [20, 64, 56, 56]          [20, 64, 28, 28]          --                        --
├─Sequential: 1-5                        [20, 64, 28, 28]          [20, 256, 28, 28]         --                        True
│    └─Bottleneck: 2-1                   [20, 64, 28, 28]          [20, 256, 28, 28]         --                        True
│    │ 

In [29]:
# for param in m_res_chk2.features.parameters():
#     param.requires_grad = False
m_res_chk2.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2),
    torch.nn.Linear(in_features=2048,
                    out_features=2, # same number of output units as our number of classes
                    bias=True),
    # torch.nn.LSTM(2048, 1,  2048),
    # torch.nn.LeakyReLU(),
    # torch.nn.Dropout(0.4)
    ).to(device)

summary(m_res_chk2, input_size=((20,3,112,112)), col_names=["input_size", "output_size", "num_params"])

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
ResNet                                   [20, 3, 112, 112]         [20, 1000]                4,098
├─Conv2d: 1-1                            [20, 3, 112, 112]         [20, 64, 56, 56]          9,408
├─BatchNorm2d: 1-2                       [20, 64, 56, 56]          [20, 64, 56, 56]          128
├─ReLU: 1-3                              [20, 64, 56, 56]          [20, 64, 56, 56]          --
├─MaxPool2d: 1-4                         [20, 64, 56, 56]          [20, 64, 28, 28]          --
├─Sequential: 1-5                        [20, 64, 28, 28]          [20, 256, 28, 28]         --
│    └─Bottleneck: 2-1                   [20, 64, 28, 28]          [20, 256, 28, 28]         --
│    │    └─Conv2d: 3-1                  [20, 64, 28, 28]          [20, 128, 28, 28]         8,192
│    │    └─BatchNorm2d: 3-2             [20, 128, 28, 28]         [20, 128, 28, 28]         256
│    │    └─ReLU: 3-3   

In [15]:
m_vgg_chk = models.vgg11(weights="IMAGENET1K_V1")
summary(m_vgg_chk, input_size=((4,3,224,224)), col_names=["input_size", "output_size", "num_params", "trainable"])

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Trainable
VGG                                      [4, 3, 224, 224]          [4, 1000]                 --                        True
├─Sequential: 1-1                        [4, 3, 224, 224]          [4, 512, 7, 7]            --                        True
│    └─Conv2d: 2-1                       [4, 3, 224, 224]          [4, 64, 224, 224]         1,792                     True
│    └─ReLU: 2-2                         [4, 64, 224, 224]         [4, 64, 224, 224]         --                        --
│    └─Conv2d: 2-3                       [4, 64, 224, 224]         [4, 64, 224, 224]         36,928                    True
│    └─ReLU: 2-4                         [4, 64, 224, 224]         [4, 64, 224, 224]         --                        --
│    └─MaxPool2d: 2-5                    [4, 64, 224, 224]         [4, 64, 112, 112]         --                        --
│    └─Co

With predef

In [37]:
class Model(nn.Module):
    def __init__(self, num_classes,latent_dim= 10240, lstm_layers=1 , hidden_dim = 10240, bidirectional = False):
        super(Model, self).__init__()
        # model = models.resnext50_32x4d(pretrained = True) #Residual Network CNN


        # VGG16

        model = models.vgg11(weights="IMAGENET1K_V1")
        # for param in model.features.parameters():
        #   param.requires_grad = False

        self.model = nn.Sequential(*list(model.children())[:-2])
        self.lstm = nn.LSTM(latent_dim,hidden_dim, lstm_layers,  bidirectional)
        self.relu = nn.LeakyReLU()
        self.dp = nn.Dropout(0.4)
        self.linear1 = nn.Linear(10240,num_classes)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
    def forward(self, x):
        batch_size,seq_length, c, h, w = x.shape
        x = x.view(batch_size * seq_length, c, h, w)
        fmap = self.model(x)
        x = self.avgpool(fmap)
        x = x.view(batch_size,seq_length,10240)
        x_lstm,_ = self.lstm(x,None)
        return fmap,self.dp(self.linear1(torch.mean(x_lstm,dim = 1)))

In [38]:
m_vgg_chk2 = Model(2)
a,b = m_vgg_chk2(torch.from_numpy(np.empty((1,20,3,112,112))).type(torch.FloatTensor))

RuntimeError: ignored

In [5]:
#Model with feature visualization
from torch import nn
from torchvision import models
class Model(nn.Module):
    def __init__(self, num_classes,latent_dim= 1280, lstm_layers=1 , hidden_dim = 1280, bidirectional = False):
        super(Model, self).__init__()
        # model = models.resnext50_32x4d(pretrained = True) #Residual Network CNN
        # wts =
        # model = models.resnext50_32x4d(wts) #Residual Network CNN
        # weights = torchvision.models.EfficientNet_B0_Weights.IMAGENET1K_V1
        # model = models.efficientnet_b0(weights)



        # VGG16

        # model = models.vgg16(weights="IMAGENET1K_V1")
        # for param in model.features.parameters():
        #   param.requires_grad = False

        # EffNet B0
        weights = torchvision.models.EfficientNet_B0_Weights.IMAGENET1K_V1
        model = models.efficientnet_b0(weights)

        self.model = nn.Sequential(*list(model.children())[:-2])
        self.lstm = nn.LSTM(latent_dim,hidden_dim, lstm_layers,  bidirectional)
        self.relu = nn.LeakyReLU()
        self.dp = nn.Dropout(0.4)
        self.linear1 = nn.Linear(1280,num_classes)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
    def forward(self, x):
        batch_size,seq_length, c, h, w = x.shape
        x = x.view(batch_size * seq_length, c, h, w)
        fmap = self.model(x)
        x = self.avgpool(fmap)
        x = x.view(batch_size,seq_length,1280)
        x_lstm,_ = self.lstm(x,None)
        return fmap,self.dp(self.linear1(torch.mean(x_lstm,dim = 1)))


# class Model(nn.Module):
#     def __init__(self, num_classes,latent_dim= 25088, lstm_layers=1 , hidden_dim = 25088, bidirectional = False):
#         super(Model, self).__init__()
#         # model = models.resnext50_32x4d(pretrained = True) #Residual Network CNN


#         # VGG16

#         model = models.vgg16(weights="IMAGENET1K_V1")
#         for param in model.features.parameters():
#           param.requires_grad = False

#         self.model = nn.Sequential(*list(model.children())[:-2])
#         self.lstm = nn.LSTM(latent_dim,hidden_dim, lstm_layers,  bidirectional)
#         self.relu = nn.LeakyReLU()
#         self.dp = nn.Dropout(0.4)
#         self.linear1 = nn.Linear(25088,num_classes)
#         self.avgpool = nn.AdaptiveAvgPool2d(1)
#     def forward(self, x):
#         batch_size,seq_length, c, h, w = x.shape
#         x = x.view(batch_size * seq_length, c, h, w)
#         fmap = self.model(x)
#         x = self.avgpool(fmap)
#         x = x.view(batch_size,seq_length,25088)
#         x_lstm,_ = self.lstm(x,None)
#         return fmap,self.dp(self.linear1(torch.mean(x_lstm,dim = 1)))

In [ ]:
# # model = Model(2)
# # a,b = model(torch.from_numpy(np.empty((1,20,3,112,112))).type(torch.FloatTensor))
# chk_vgg = Model(2)
# a,b = chk_vgg(torch.from_numpy(np.empty((1,20,3,112,112))).type(torch.FloatTensor))

In [9]:
import numpy as np
chk_eff0 = Model(2)
a,b = chk_eff0(torch.from_numpy(np.empty((1,20,3,112,112))).type(torch.FloatTensor))

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


In [10]:
summary(chk_eff0, input_size=((1,20,3,112,112)), col_names=["input_size", "output_size", "num_params", "trainable"])

Layer (type:depth-idx)                                       Input Shape               Output Shape              Param #                   Trainable
Model                                                        [1, 20, 3, 112, 112]      [20, 1280, 4, 4]          --                        True
├─Sequential: 1-1                                            [20, 3, 112, 112]         [20, 1280, 4, 4]          --                        True
│    └─Sequential: 2-1                                       [20, 3, 112, 112]         [20, 1280, 4, 4]          --                        True
│    │    └─Conv2dNormActivation: 3-1                        [20, 3, 112, 112]         [20, 32, 56, 56]          928                       True
│    │    └─Sequential: 3-2                                  [20, 32, 56, 56]          [20, 16, 56, 56]          1,448                     True
│    │    └─Sequential: 3-3                                  [20, 16, 56, 56]          [20, 24, 28, 28]          16,714            

In [ ]:
m_qu = torchvision.models.quantization.googlenet(weights="IMAGENET1K_FBGEMM_V1", quantize=True)
summary(m_qu, input_size=((4,3,224,224)), col_names=["input_size", "output_size", "num_params", "trainable"])